# Symbolic neural networks for cognitive capacities
[Paper](http://reason.cs.uiuc.edu/tsvi/BICA_93_Main.pdf)

In [1]:
%run achler_functions.py

In [2]:
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog')[0]
dog

Synset('dog.n.01')

In [3]:
dog.hypernyms()

[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

In [4]:
print(dog.name().split(".")[0])
dog_hypernyms = [hypernym.name().split(".")[0] for hypernym in dog.hypernyms()]
dog_hypernyms

dog


['canine', 'domestic_animal']

In [5]:
dogs = [hyponym.name().split(".")[0] for hyponym in dog.hyponyms()]
dogs

['basenji',
 'corgi',
 'cur',
 'dalmatian',
 'great_pyrenees',
 'griffon',
 'hunting_dog',
 'lapdog',
 'leonberg',
 'mexican_hairless',
 'newfoundland',
 'pooch',
 'poodle',
 'pug',
 'puppy',
 'spitz',
 'toy_dog',
 'working_dog']

In [6]:
cat = wn.synsets('cat')[0]
cats = [hyponym.name().split(".")[0] for hyponym in cat.hyponyms()]
cats

['domestic_cat', 'wildcat']

In [7]:
cat_hypernyms = [hypernym.name().split(".")[0] for hypernym in cat.hypernyms()]
cat_hypernyms.append('domestic_animal')
cat_hypernyms

['feline', 'domestic_animal']

In [8]:
cat = wn.synsets('domestic_cat')[0]
cats = set([hyponym.name().split(".")[0] for hyponym in cat.hyponyms()])
cats

{'abyssinian',
 'alley_cat',
 'angora',
 'burmese_cat',
 'egyptian_cat',
 'kitty',
 'maltese',
 'manx',
 'mouser',
 'persian_cat',
 'siamese_cat',
 'tabby',
 'tiger_cat',
 'tom',
 'tortoiseshell'}

# Learn dogs and cats dataset instances

In [9]:
from sklearn import datasets
import numpy as np

In [10]:
exp = np.zeros( (len(dogs) + len(cats), len(dogs) + len(dog_hypernyms) + len(cats) + len(cat_hypernyms)) )
exp_labels = []

# dogs
for item in dogs:
    instance = {}
    instance = {item: 1}
    for hypernym in dog_hypernyms:
        instance[hypernym] = 1
    exp, exp_labels = learn_instance([instance,], exp, exp_labels)

# cats
for item in cats:
    instance = {}
    instance = {item: 1}
    for hypernym in cat_hypernyms:
        instance[hypernym] = 1
    exp, exp_labels = learn_instance([instance,], exp, exp_labels)
    
print(exp.shape)
print(exp[:2])
print("...")
print(exp[-2:])
print(exp_labels)

(33, 37)
[[ 1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]]
...
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
   0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.]]
['basenji', 'domestic_animal', 'canine', 'corgi', 'cur', 'dalmatian', 'great_pyrenees', 'griffon', 'hunting_dog', 'lapdog', 'leonberg', 'mexican_hairless', 'newfoundland', 'pooch', 'poodle', 'pug', 'puppy', 'spitz', 'toy_dog', 'working_dog', 'manx', 'feline', 'mouser', 'kitty', 'burmese_cat', 'angora', 'tabby', 'alley_cat', 'siamese_cat', 'tom', 'tortoiseshell', 

## Make categories of the common types

In [11]:
category_list = [exp_labels[i] for i in np.where(exp.sum(axis=0) > 1)[0]] # more than one row
category_list

['domestic_animal', 'canine', 'feline']

In [12]:
def make_categories(category_list, exp, exp_labels):
    categories = {}
    for category in category_list:
        if category in exp_labels:
            category_index = exp_labels.index(category)
            categories[category] = np.matrix(np.zeros(exp.shape[1]))
            for index in range(exp.shape[0]):
                if exp.item( (index, category_index) ) != 0:
                    categories[category] += exp[index, :]
    return categories
categories = make_categories(category_list, exp, exp_labels)
categories

{'canine': matrix([[  1.,  18.,  18.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
            1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.]]),
 'domestic_animal': matrix([[  1.,  33.,  18.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
            1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,  15.,
            1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
            1.,   1.,   1.,   0.]]),
 'feline': matrix([[  0.,  15.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,  15.,
            1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,
            1.,   1.,   1.,   0.]])}

Do the categories work?

In [13]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()
prediction_canine = pinv_transpose * categories["canine"].T
prediction_canine.round(2).T

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [14]:
prediction_feline = pinv_transpose * categories["feline"].T
prediction_feline.round(2).T

array([[ 0.,  0., -0.,  0.,  0., -0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [15]:
prediction_domestic_animal = pinv_transpose * categories["domestic_animal"].T
prediction_domestic_animal.round(2).T

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.]])

Which is the biggest category? Can we tell the others are subcategories?

In [16]:
for category in categories:
    print(category, np.sum(categories[category]))

feline 45.0
domestic_animal 99.0
canine 54.0


In [17]:
np.all(categories['domestic_animal'] == categories['canine'] + categories['feline'])

True